In [1]:
using Pkg
# Pkg.add("Random")
# Pkg.add("Plots")
# Pkg.add("LaTeXStrings")
# Pkg.add("Statistics")
# Pkg.add("CSV")
# Pkg.add("DataFrames")
# Pkg.add("LoopVectorization")
# Pkg.add("ThreadsX")
# Pkg.add("GLM")
# Pkg.add("SparseArrays)
# Pkg.add("SpecialFunctions")
# Pkg.add("JLD2")

In [3]:
using Random
using Plots
using LaTeXStrings
using Statistics
using CSV
using DataFrames
using Base.Threads       # For multi-threading
using LoopVectorization  # For increased performance
using GLM
using SparseArrays
# using SpecialFunctions
using JLD2

In [ ]:
function get_root_node(i::Int, network)
    if network[i] < 0  # Check if i is the root node
        return i
    else
        # Path compression: set parent directly to root for efficiency
        network[i] = get_root_node(network[i], network)
        return network[i]
    end
end

function activate_bond(k, network, lattice_matrix, cluster_dict, greatest_cluster_ref, avg_s)
    i = lattice_matrix[k, 1]  # Node 1
    j = lattice_matrix[k, 2]  # Node 2

    root_i = get_root_node(i, network)
    root_j = get_root_node(j, network)

    if root_i != root_j
        # Ensure the larger cluster remains the root
        if abs(network[root_i]) < abs(network[root_j])
            root_i, root_j = root_j, root_i  # Swap to make root_i the larger cluster
        end

        avg_s = avg_s - network[root_i]^2 - network[root_j]^2 + (network[root_i] + network[root_j])^2  # New size of the merged cluster
        # Merge root_j into root_i
        network[root_i] -= abs(network[root_j])  # Update size of the merged cluster

        # Update all nodes in cluster_j to point to root_i
        if haskey(cluster_dict, root_j)
            for node in cluster_dict[root_j]
                network[node] = root_i  # Update parent
            end
            # Merge the node lists
            cluster_dict[root_i] = vcat(cluster_dict[root_i], cluster_dict[root_j])
            delete!(cluster_dict, root_j)  # Remove the old cluster entry
        end

        # Update the greatest cluster reference if necessary
        if abs(network[root_i]) > abs(network[greatest_cluster_ref[]])
            greatest_cluster_ref[] = root_i
        end
                
    end
    return avg_s
end

function lattice_loader(file_name::String)
    if !isfile(file_name)
        error("File $file_name does not exist")
    end
    lattice = []
    open(file_name, "r") do file_name
        for line in eachline(file_name)
            push!(lattice, parse(Int, line))
        end
    end
    lattice_matrix = reshape(lattice, 2, :)'
end

function shuffle_bonds(lattice_matrix, rng)
    M = size(lattice_matrix, 1)
    for i in 2:M  
        r = rand(rng, i:M) 
        lattice_matrix[[i-1, r], :] = lattice_matrix[[r, i-1], :]
    end
    return lattice_matrix
end


In [ ]:
# const log_fact_dict = Dict{Int, Float64}()  # Store log factorials for memoization
# const binom_coeff = Dict{Int, Float64}()  # Store binomial coefficients for memoization
# const log_fact_lock = ReentrantLock()
# const binom_lock = ReentrantLock()

# function log_factorial(n::Int)
#     if n < 0
#         throw(ArgumentError("n must be non-negative"))
#     elseif haskey(log_fact_dict, n)
#         return log_fact_dict[n]
#     else
#         result = sum(log.(1:n))
        
#         lock(log_fact_lock) do
#             log_fact_dict[n] = result
#         end 
#         return result
#     end
# end

# function logbinom(n::Int, k::Int)
#     if k < 0 || k > n
#         return -Inf   # Invalid coeff
#     end
#     return log_factorial(n) - log_factorial(k) - log_factorial(n-k)
# end

# function precompute_log_factorials(max_n::Int)
#     @threads for i in 0:max_n
#         log_factorial(i)
#     end
# end

# function compute_binom_coeffs(M::Int)
#     for n in 1:M
#         result = logbinom(M, n)
        
#         # Thread-safe dictionary update
#         lock(binom_lock) do
#             binom_coeff[n] = result
#         end
#     end
# end

function binom_coeff_row(m::Int)
    row = Vector{BigInt}(undef, m+1)
    row[1] = BigInt(1)  # C(0, 0) = 1
    for n in 1:m
        row[n+1] = row[n] * (m - n + 1) ÷ n
    end
    return row
end

function save_binomial_jld2(filename::String, ms::Vector{Int})
    JLD2.jldopen(filename, "w") do file  # Open the file in write mode
        file["ms"] = ms  # Save the list of M values
        for m in ms
            row = binom_coeff_row(m)
            file["row_$m"] = row  # Save each row under a unique key
        end
    end
end

function load_binomial_row(filename::String, m::Int)
    return JLD2.load(filename, "row_$m")
end

In [ ]:
# Main function

function percolater(lattice_matrix, N, M)
    avg_s_val = N                    # Initial value for avgerage_s (sum of the size squared for all clusters)
    network = fill(Int(-1), N)       # Initialize network with all nodes as root nodes
    shuffle_bonds(lattice_matrix, rng)    # Shuffle bonds
    print(lattice_matrix)
    # prepare cluster dictionary (dict with root node indeces as keys and list of nodes in cluster as values)
    cluster_dict = Dict{Int, Vector{Int}}()
    for i in 1:N
        cluster_dict[i] = [i]
    end

    # Pointer for saving values from loop of activate_bonds
    greatest_cluster_ref = Ref(1)
    k_max = Ref(1)

    # Arrays to save quantities of interest
    P_inf = zeros(M)
    avg_s = ones(M)*N

    # Iterating over all bonds
    for k in 1:M 
        
        # Activating bonds
        avg_s_val = activate_bond(k, network, lattice_matrix, cluster_dict, greatest_cluster_ref, avg_s_val)
        P_inf[k] = abs(network[greatest_cluster_ref[]]) / N
        avg_s[k] = avg_s_val
        
        # Save the biggest cluster evently ten times evenly to plot
        T = M/10
        if k%(T) == 0
            greatest_cluster = greatest_cluster_ref[]
            
            # println("greatest: ", greatest_cluster)
            cluster_nodes = cluster_dict[greatest_cluster]
            push!(pictures, cluster_nodes)

        end

        # Cut off if all nodes are in the same cluster
        if abs(network[greatest_cluster_ref[]]) == N
            k_max[] = k    # Cut off, when all sites are in the same cluster
            println(k_max[])
            break
        end
    end 

    # Fill elements after the cut off with the last value, since no change will occur
    P_inf[k_max[]+1:end] .= P_inf[k_max[]]    # Cut off rest of the array
    avg_s[k_max[]+1:end] .= avg_s[k_max[]]    # Cut off rest of the array
    
    average_cluster_size = (avg_s .- (N*P_inf).^2) ./ (N*(1 .- P_inf))    # <s>

    indicies_to_change = findall(P_inf .> 0.9999)      # Avoid <s>->\infty when P_inf = 1
    average_cluster_size[ indicies_to_change] .= 0  

    return P_inf, average_cluster_size
end

In [ ]:
# Generating lattice

# Parameters
L = 100  # Number of nodes in one dimension
N = L^2  # Number of nodes
type = "square"

# Function for dynamical file name registration
function format_with_underscores(n::Int)
    s = string(n)
    reversed_chunks = reverse.(Iterators.partition(reverse(s), 3))
    return join(reverse(reversed_chunks), '_')
end

N_str = format_with_underscores(N) * "_$type"
file_name = "lattice_$N_str.txt"
println(file_name)

# Load lattice and reshape into matrix
lattice_matrix = lattice_loader(file_name)

M = size(lattice_matrix)[1]  # Number of bonds

In [ ]:
# RNG generator

seed = 123
rng = Xoshiro(seed) # sjekk om denne er bra) 
# rng = MersenneTwister(seed)

In [ ]:
# Calculate the binomial coefficients

precompute_log_factorials(M)
compute_binom_coeffs(M)

println(length(binom_coeff))

In [ ]:
# Main

num_it = 100   # Number of iterations of the percolater for averaging

pictures = []  # Array for saving plots of the biggest cluster

P_arr = zeros(num_it, M)                # Array for saving P_inf    
average_cluster_size = zeros(num_it, M) # Array for saving <s>


# Multiple iterations of the percolater
for l in 1:num_it
    
    P_inf, avg_s = percolater(lattice_matrix, N, M)
    P_arr[l, :] = P_inf
    average_cluster_size[l, :] = avg_s   

end


## Plotting pictures

In [ ]:
function to_coords(i::Int, L::Int)
    """
    i (int): Index of node
    L (int): Number of nodes in each row
    """
    y = i%L
    if y == 0
        y = L
    end
   
    x = div(i, L) +1
    if i % L == 0
       x = div(i, L)
    end
    return [x, y]
end

## NBNBNB byttet om pga julia kolonnestyr

In [ ]:
function plot_picture(cluster, L)
    """
    cluster (array): List of nodes in cluster
    L (int): Number of nodes in each row
    """
    grid = zeros(Int, L, L)
    for node in cluster
        coords = to_coords(node, L)
        grid[coords[1], coords[2]] = 1
    end
    grid = reverse(grid, dims=1)
    return grid
end

In [ ]:
heatmaps = []
for i in 1:9
    grid = plot_picture(pictures[i], L)
    p_values = i/10
    push!(heatmaps, heatmap(grid, title="p=$p_values"))
end

# Arrange the heatmaps in a 3x3 grid
plot(heatmaps..., layout=(3,3), size=(800, 500))

In [ ]:
P_avg = mean(P_arr, dims=1)
P_2_avg = mean(P_arr.^2, dims=1)

chi = N*sqrt.(P_2_avg .- P_avg.^2)
p = (1:M) / M   # probability of activating a node, with cut-off

plot(p, P_avg', xlabel="p", xticks=(0:0.1:1.0), ylabel=L"\langle P_\infty\rangle", title=L"Giant component, $P_\infty$", label="N=$N", legend=:topleft)
# plot!(p, chi, xlabel="p", ylabel="χ", title="Susceptibility", label=L"\chi", legend=:topleft)

In [ ]:
plot(p, chi', xlabel="p", xticks=(0:0.1:1.0), ylabel=L"\chi", title=L"Susceptibility, $\chi$", label="N=$N", legend=:topleft)


In [ ]:
average_cluster_size_avg = mean(average_cluster_size, dims=1)

plot(p, average_cluster_size_avg', xlabel="p", xticks=(0:0.1:1.0), ylabel=L"\langle s \rangle", title=L"Average cluster size, $\langle s \rangle$", label="N=$N", legend=:topleft)


In [ ]:
Threads.nthreads()

In [ ]:
# Convolution
num_it_q = 10_000
q = range(0.0, 1.0, length=num_it_q)

P_conv = zeros(length(q))
P_2_conv = zeros(length(q))
average_cluster_size_conv = zeros(length(q))

log_q = log.(q)
log1_q = log1p.(-q)

binom_coeff_array = [binom_coeff[n] for n in 1:M]  # Convert Dict to Array

for i in 1:length(q)
    # P_conv[i] = 0.0
    # average_cluster_size_conv[i] = N
    # for n in 1:M
    #     log_P = binom_coeff[n] + n*log(q[i]) + (M-n)*log1p(- q[i]) + log(P_avg[1, n]) #log1p for better stability when 1-q[i]->1
    #     log_P_2 = binom_coeff[n] + n*log(q[i]) + (M-n)*log1p(- q[i]) + log(P_2_avg[1, n])
    #     log_s = binom_coeff[n] + n*log(q[i]) + (M-n)*log1p(- q[i]) + log(average_cluster_size_avg[1, n])

    #     P_conv[i] += exp(log_P)
    #     P_2_conv[i] += exp(log_P_2)
    #     average_cluster_size_conv[i] += exp(log_s)
    # end
    P_conv[i] = sum(exp.(binom_coeff_array[1:M] .+ (1:M) .* log_q[i] .+ (M .- (1:M)) .* log1_q[i] .+ log.(P_avg[1, 1:M])))
    P_2_conv[i] = sum(exp.(binom_coeff_array[1:M] .+ (1:M) .* log_q[i] .+ (M .- (1:M)) .* log1_q[i] .+ log.(P_2_avg[1, 1:M])))
    average_cluster_size_conv[i] = sum(exp.(binom_coeff_array[1:M] .+ (1:M) .* log_q[i] .+ (M .- (1:M)) .* log1_q[i] .+ log.(average_cluster_size_avg[1, 1:M])))

end

In [ ]:
plot(q,P_conv, xlabel="q", xticks=(0:0.1:1.0), ylabel=L"\langle P_\infty\rangle", title=L"Giant component, $P_\infty$", label="N=$N", legend=:topleft)


In [ ]:
# display(average_cluster_size_conv)
plot(q, average_cluster_size_conv, xlabel="q", xticks=(0:0.1:1.0), ylabel=L"\langle s \rangle", title=L"average_cluster_size, $\langle s \rangle$", label="N=$N", legend=:topleft)


In [ ]:
argument = P_2_conv .- P_conv.^2
argument[findall(argument .< 0)] .= 0      # Avoid all zero values

chi_conv = N*sqrt.(argument)

df = DataFrame(q=q, P=P_conv, chi=chi_conv, average_cluster_size=average_cluster_size_conv)
N_str = format_with_underscores(N)
q_str = format_with_underscores(num_it_q)
CSV.write("N$(N_str)_q$(q_str)_$(type).csv", df)

In [ ]:
plot(q, chi_conv, xlabel="q", xticks=(0:0.1:1.0), ylabel=L"\langle s \rangle", title=L"Susceptibility, $\chi$", label="N=$N", legend=:topleft)


## Data analysis

### Square lattice

In [4]:
df_10_000_square = CSV.read("DataFrames/N10_000_q10_000_square.csv", DataFrame)
df_40_000_square = CSV.read("DataFrames/N40_000_q10_000_square.csv", DataFrame)
df_90_000_square = CSV.read("DataFrames/N90_000_q10_000_square.csv", DataFrame)
df_160_000_square = CSV.read("DataFrames/N160_000_q10_000_square.csv", DataFrame)
df_250_000_square = CSV.read("DataFrames/N250_000_q10_000_square.csv", DataFrame)
df_360_000_square = CSV.read("DataFrames/N360_000_q10_000_square.csv", DataFrame)
df_490_000_square = CSV.read("DataFrames/N490_000_q10_000_square.csv", DataFrame)
df_640_000_square = CSV.read("DataFrames/N640_000_q10_000_square.csv", DataFrame)
df_810_000_square = CSV.read("DataFrames/N810_000_q10_000_square.csv", DataFrame)
df_1_000_000_square = CSV.read("DataFrames/N1_000_000_q10_000_square.csv", DataFrame)

Row,q,P,chi,average_cluster_size
,Float64,Float64,Float64,Float64
1,0.0,4.00001e-12,0.00282843,1.0
2,0.00010001,2.07787e-6,0.267918,1.0004
3,0.00020002,2.23632e-6,0.424792,1.0008
4,0.00030003,2.43965e-6,0.49632,1.0012
5,0.00040004,2.59991e-6,0.489892,1.0016
6,0.00050005,2.79946e-6,0.400374,1.002
7,0.00060006,2.9019e-6,0.297414,1.0024
8,0.00070007,2.95831e-6,0.199812,1.0028
9,0.00080008,2.99898e-6,0.0315194,1.00321


In [5]:
FSS_P_q_square = plot(df_10_000_square.q[4000:6000], df_10_000_square.P[4000:6000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=10_000", legend=:topleft)
plot!(FSS_P_q_square, df_40_000_square.q[4000:6000], df_40_000_square.P[4000:6000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=40_000", legend=:topleft)
plot!(FSS_P_q_square,df_90_000_square.q[4000:6000], df_90_000_square.P[4000:6000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=90_000", legend=:topleft)
plot!(FSS_P_q_square,df_160_000_square.q[4000:6000], df_160_000_square.P[4000:6000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=160_000", legend=:topleft)
plot!(FSS_P_q_square,df_250_000_square.q[4000:6000], df_250_000_square.P[4000:6000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=250_000", legend=:topleft)
plot!(FSS_P_q_square,df_360_000_square.q[4000:6000], df_360_000_square.P[4000:6000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=360_000", legend=:topleft)
plot!(FSS_P_q_square,df_490_000_square.q[4000:6000], df_490_000_square.P[4000:6000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=490_000", legend=:topleft)
plot!(FSS_P_q_square,df_640_000_square.q[4000:6000], df_640_000_square.P[4000:6000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=640_000", legend=:topleft)
plot!(FSS_P_q_square,df_810_000_square.q[4000:6000], df_810_000_square.P[4000:6000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=810_000", legend=:topleft)
plot!(FSS_P_q_square,df_1_000_000_square.q[4000:6000], df_1_000_000_square.P[4000:6000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=1_000_000", legend=:topleft)
savefig(FSS_P_q_square, "figures/FSS_P_q_square.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_P_q_square.svg"

In [6]:
# FSS 

ksi = round.(Int, sqrt.([10_000, 40_000, 90_000, 160_000, 250_000, 360_000, 490_000, 640_000, 810_000, 1_000_000]))

df_P_square = DataFrame(q = df_10_000_square.q)  # Initialize with q values (common for all N)
df_P_square[!, "P_ksi_$(ksi[1])"] = df_10_000_square.P
df_P_square[!, "P_ksi_$(ksi[2])"] = df_40_000_square.P
df_P_square[!, "P_ksi_$(ksi[3])"] = df_90_000_square.P
df_P_square[!, "P_ksi_$(ksi[4])"] = df_160_000_square.P
df_P_square[!, "P_ksi_$(ksi[5])"] = df_250_000_square.P
df_P_square[!, "P_ksi_$(ksi[6])"] = df_360_000_square.P
df_P_square[!, "P_ksi_$(ksi[7])"] = df_490_000_square.P
df_P_square[!, "P_ksi_$(ksi[8])"] = df_640_000_square.P
df_P_square[!, "P_ksi_$(ksi[9])"] = df_810_000_square.P
df_P_square[!, "P_ksi_$(ksi[10])"] = df_1_000_000_square.P


# df_P_square = df_P_square[2:end-1, :]  # Remove NaN in the first and last row
# plt = plot(; xscale=:log10, yscale=:log10, xlabel="q", ylabel=L"\langle P_\infty\rangle", title=L"Giant component, $P_\infty$")

function line_fitter(df, ksi)
    r2_arr = zeros(size(df,1))
    model_arr = []
    slope_arr = []

    for i in 1:size(df, 1)
        data_row = Vector(df[i, 2:end])    # Exclude q
        df_row = DataFrame(log_ksi=log.(ksi), log_param=log.(data_row))
        model = lm(@formula(log_param ~ log_ksi), df_row)
        r2_arr[i] = r2(model)

        push!(model_arr, model)
        push!(slope_arr, coef(model)[2])
    end
    return r2_arr, model_arr, slope_arr
end

r2_arr_P_square, model_arr_P_square, slope_arr_P_square = line_fitter(df_P_square, ksi)

FFS_P_R2_square = plot(df_P_square.q, r2_arr_P_square, xlabel="q", xticks=0:0.1:1, yticks=0:0.1:1, ylabel=L"$R^2$-coefficient", color=:green, legend=false)
savefig(FFS_P_R2_square, "figures/FFS_P_R2_square.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FFS_P_R2_square.svg"

In [7]:
i_start = 4900
i_end = 5100
# dr2_arr = diff(r2_arr_P_square)
# println(model_arr_P[])
# println(argmin(r2_arr_P_square[5000:5100]))
# println(slope_arr_P[5000+argmax(r2_arr_P_square[5050:5100])])
best_index_P_square = i_start+argmax(r2_arr_P_square[i_start:i_end]) - 1 # -1 since argmax finds from 4900, so no need for double counting
println(best_index_P_square)
# p_inset = plot(df_P_square.q[i_start:i_end], r2_arr_P_square[i_start:i_end], framestyle=:box)

# plot!(ones(i_end - i_start)*best_index, , DataFrame(log_ksi=log.(ksi))))

FFS_P_R2_square_zoomed = plot(df_P_square.q[i_start:i_end], r2_arr_P_square[i_start:i_end], xlabel="q",xticks=0.4:0.002:0.6, yticks=0:0.1:1, ylabel=L"$R^2$-coefficient", color=:green, label=L"Square: $R^2$")
# plot!(ones(i_end - i_start)*df_P_square.q[best_index], 0:0.01:1, linestyle=:dash, color=:black, label="Best fit")
vline!(FFS_P_R2_square_zoomed, [df_P_square.q[best_index_P_square]], color=:red, linestyle=:dash, label=L"$p_c$")
savefig(FFS_P_R2_square_zoomed, "figures/FFS_P_R2_square_zoomed.svg")

5012


"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FFS_P_R2_square_zoomed.svg"

In [8]:
p_c = df_P_square.q[best_index_P_square]
println("p_c: ", p_c)

p_c: 0.5011501150115012


In [9]:
FSS_s_square = plot(df_10_000_square.q[4500:5300], df_10_000_square.average_cluster_size[4500:5300], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=10_000", legend=:topright)
plot!(FSS_s_square, df_40_000_square.q[4500:5300], df_40_000_square.average_cluster_size[4500:5300], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=40_000")
plot!(FSS_s_square,df_90_000_square.q[4500:5300], df_90_000_square.average_cluster_size[4500:5300], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=90_000")
plot!(FSS_s_square,df_160_000_square.q[4500:5300], df_160_000_square.average_cluster_size[4500:5300], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=160_000")
plot!(FSS_s_square,df_250_000_square.q[4500:5300], df_250_000_square.average_cluster_size[4500:5300], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=250_000")
plot!(FSS_s_square,df_360_000_square.q[4500:5300], df_360_000_square.average_cluster_size[4500:5300], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=360_000")
plot!(FSS_s_square,df_490_000_square.q[4500:5300], df_490_000_square.average_cluster_size[4500:5300], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=490_000")
plot!(FSS_s_square,df_640_000_square.q[4500:5300], df_640_000_square.average_cluster_size[4500:5300], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=640_000")
plot!(FSS_s_square,df_810_000_square.q[4500:5300], df_810_000_square.average_cluster_size[4500:5300], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=810_000")
plot!(FSS_s_square,df_1_000_000_square.q[4500:5300], df_1_000_000_square.average_cluster_size[4500:5300], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=1_000_000")
savefig(FSS_s_square, "figures/FSS_s_square.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_s_square.svg"

In [10]:
# FSS - average cluster 

df_s_square = DataFrame(q = df_10_000_square.q)  # Initialize with q values (common for all N)
df_s_square[!, "s_ksi_$(ksi[1])"] = df_10_000_square.average_cluster_size
df_s_square[!, "s_ksi_$(ksi[2])"] = df_40_000_square.average_cluster_size
df_s_square[!, "s_ksi_$(ksi[3])"] = df_90_000_square.average_cluster_size
df_s_square[!, "s_ksi_$(ksi[4])"] = df_160_000_square.average_cluster_size
df_s_square[!, "s_ksi_$(ksi[5])"] = df_250_000_square.average_cluster_size
df_s_square[!, "s_ksi_$(ksi[6])"] = df_360_000_square.average_cluster_size
df_s_square[!, "s_ksi_$(ksi[7])"] = df_490_000_square.average_cluster_size
df_s_square[!, "s_ksi_$(ksi[8])"] = df_640_000_square.average_cluster_size
df_s_square[!, "s_ksi_$(ksi[9])"] = df_810_000_square.average_cluster_size
df_s_square[!, "s_ksi_$(ksi[10])"] = df_1_000_000_square.average_cluster_size

# df_s_square = df_s_square[2:end-1, :]  # Remove NaN in the first and last row
max_vec = zeros(length(ksi))
for i in 1:length(ksi)
    max_vec[i] = maximum(df_s_square[!, "s_ksi_$(ksi[i])"])
end
println(max_vec)
df_sm_square = DataFrame(ksi=ksi)  # Initialize with q values (common for all N)
df_sm_square[!, "max_s"] = max_vec

display(df_sm_square)
# threshold = 1e-6   # Threshold all values below this to zero

# df_s_square = df_s_square[2:end-1, :]  # Remove NaN in the first row
# for col in names(df_s_square)
#     df_s_square[!, col] = map(x -> x < threshold ? 0 : x, df_s_square[!, col])
# end

# r2_arr_s_square, model_arr_s_square, slope_arr_s_square = line_fitter(df_sm_square, ksi)

Row,ksi,max_s
,Int64,Float64
1,100,313.659
2,200,1098.77
3,300,2116.72
4,400,3878.62
5,500,5652.23
6,600,7777.93
7,700,9095.26
8,800,12148.6
9,900,16239.0


[313.6590895803622, 1098.7739955355112, 2116.719978626081, 3878.617139043189, 5652.225946649163, 7777.930309010547, 9095.260175408823, 12148.57309045354, 16238.958036079488, 18768.586404553847]


In [11]:
model_sm_square = lm(@formula(log(max_s) ~ log(ksi)), df_sm_square)
println(r2(model_sm_square))
display(model_sm_square)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

:(log(max_s)) ~ 1 + :(log(ksi))

Coefficients:
─────────────────────────────────────────────────────────────────────────
                Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────
(Intercept)  -2.39909   0.132842   -18.06    <1e-07   -2.70542   -2.09275
log(ksi)      1.77115   0.0215827   82.06    <1e-12    1.72138    1.82092
─────────────────────────────────────────────────────────────────────────

0.9988134791616269


In [12]:
FSS_sm_square = scatter(log.(df_sm_square.ksi), log.(df_sm_square.max_s), xlabel=L"log(\xi)", ylabel=L"log(max(\langle s \rangle))", color=:green, label=L"Square: $max(\langle s \rangle))$ ", legend=:topleft)
plot!(FSS_sm_square , log.(df_sm_square.ksi), predict(model_sm_square), color=:red, label="Best fit")
annotate!(FSS_sm_square, 4.9, 9.1, text("R^2: $(round(r2(model_sm_square),digits=4))",8))
savefig(FSS_sm_square, "figures/FSS_sm_square.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_sm_square.svg"

In [335]:
# i_start = 5000
# i_end = 6000
# best_index_s_square = i_start+argmax(r2_arr_s_square[i_start:i_end])
# println(i_start+argmax(r2_arr_s_square[i_start:i_end]))
# plot(df_s_square.q[i_start:i_end], r2_arr_s_square[i_start:i_end], xlabel="q", xticks=0.4:0.002:0.6, ylabel=L"$R^2$-coefficient", color=:green, label=L"$R^2$", legend=:topright)
# vline!([df_s_square.q[best_index_s_square]], color=:red, linestyle=:dash, label=L"$p_c$")
# display(model_arr_s_square[best_index_s_square])

In [13]:
# Chi
FSS_chi_square = plot(df_10_000_square.q[4500:5500], df_10_000_square.chi[4500:5500], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=10_000", legend=:topright)
plot!(FSS_chi_square, df_40_000_square.q[4500:5500], df_40_000_square.chi[4500:5500], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=40_000")
plot!(FSS_chi_square,df_90_000_square.q[4500:5500], df_90_000_square.chi[4500:5500], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=90_000")
plot!(FSS_chi_square,df_160_000_square.q[4500:5500], df_160_000_square.chi[4500:5500], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=160_000")
plot!(FSS_chi_square,df_250_000_square.q[4500:5500], df_250_000_square.chi[4500:5500], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=250_000")
plot!(FSS_chi_square,df_360_000_square.q[4500:5500], df_360_000_square.chi[4500:5500], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=360_000")
plot!(FSS_chi_square,df_490_000_square.q[4500:5500], df_490_000_square.chi[4500:5500], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=490_000")
plot!(FSS_chi_square,df_640_000_square.q[4500:5500], df_640_000_square.chi[4500:5500], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=640_000")
plot!(FSS_chi_square,df_810_000_square.q[4500:5500], df_810_000_square.chi[4500:5500], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=810_000")
plot!(FSS_chi_square,df_1_000_000_square.q[4500:5500], df_1_000_000_square.chi[4500:5500], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=1_000_000")
savefig(FSS_chi_square, "figures/FSS_chi_square.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_chi_square.svg"

In [14]:
## FSS - ksi
q_max = zeros(length(ksi))
for i in 1:length(ksi)
    q_max[i] = df_s_square.q[argmax(df_s_square[!, "s_ksi_$(ksi[i])"])]
end
println(q_max)


df_max_square = DataFrame(ksi=ksi)  # Initialize with q values (common for all N)
df_max_square[!, "q_max"] = abs.(q_max.- p_c)
display(df_max_square)
# r2_arr_max_square, model_arr_max_square, slope_arr_max_square = line_fitter(df_max_square, ksi)


Row,ksi,q_max
,Int64,Float64
1,100,0.0185019
2,200,0.010401
3,300,0.00880088
4,400,0.00730073
5,500,0.00460046
6,600,0.0050005
7,700,0.00490049
8,800,0.00450045
9,900,0.00420042


[0.48264826482648265, 0.49074907490749076, 0.49234923492349236, 0.49384938493849384, 0.4965496549654965, 0.4961496149614962, 0.49624962496249625, 0.49664966496649665, 0.4969496949694969, 0.49744974497449745]


In [15]:
model_max_square = lm(@formula(log(q_max) ~ log(ksi) ), df_max_square)
println(r2(model_max_square))
display(model_max_square)

0.9640632533926495


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

:(log(q_max)) ~ 1 + :(log(ksi))

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  -0.907883   0.285818    -3.18    0.0131  -1.56698   -0.248786
log(ksi)     -0.68028    0.0464365  -14.65    <1e-06  -0.787363  -0.573197
──────────────────────────────────────────────────────────────────────────

In [16]:
FSS_max_square = scatter(log.(ksi), log.(df_max_square.q_max), xlabel=L"log(\xi)", ylabel=L"log(\left|q_{max}-p_c\right|)",color=:green, label="Square: q_max",legend=:topright)
plot!(FSS_max_square, log.(ksi), predict(model_max_square), color=:red, label="Best fit")
annotate!(FSS_max_square, 6.5, -4.25, text("R^2: $(round(r2(model_max_square),digits=4))",8))
savefig(FSS_max_square, "figures/FSS_max_square.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_max_square.svg"

In [340]:
# i_start = 4500
# i_end = 5100

# best_index_max_square = i_start+argmax(r2_arr_max_square[i_start:i_end])
# println(best_index_max_square)

# plot(df_max_square.q[i_start:i_end], r2_arr_max_square[i_start:i_end], xlabel="q", xticks=0:0.01:1, ylabel=L"$R^2$-coefficient", color=:green, legend=false)
# vline!([df_max_square.q[best_index_max_square]], color=:red, linestyle=:dash, label=L"$p_c$")

In [341]:
# println(model_arr_max_square[best_index_max_square])
# println(model_arr_max_square[5018])

In [342]:
# println("Best index, P: ", best_index_P_square)
# println("Best index, s: ", best_index_s_square)
# println("Best index, max: ", best_index_max_square)

In [17]:
# Upper and lower bounds

lower_P_square = coeftable(model_arr_P_square[best_index_P_square]).cols[5][2]
upper_P_square = coeftable(model_arr_P_square[best_index_P_square]).cols[6][2]

lower_s_square = coeftable(model_sm_square).cols[5][2]
upper_s_square = coeftable(model_sm_square).cols[6][2]

lower_max_square = coeftable(model_max_square).cols[5][2]
upper_max_square = coeftable(model_max_square).cols[6][2]

println(lower_P_square, " ", upper_P_square, " ", lower_s_square, " ", upper_s_square, " ", lower_max_square, " ", upper_max_square)

-0.07911976056792722 -0.060872672338460464 1.7213764022251195 1.820915788245178 -0.7873629900324879 -0.5731974718370301


In [18]:
nu_square = - 1 / coef(model_max_square)[2]
gamma_square = nu_square*coef(model_sm_square)[2]
beta_square = -nu_square*slope_arr_P_square[best_index_P_square]
println("Numerical: nu = $(nu_square), gamma = $(gamma_square), beta = $(beta_square)")
println("Analytical: nu = $(4/3), gamma = $(43/18), beta = $(5/36)")

Numerical: nu = 1.4699824491826268, gamma = 2.60355367493401, beta = 0.10289320969538317
Analytical: nu = 1.3333333333333333, gamma = 2.388888888888889, beta = 0.1388888888888889


In [19]:
nu_upper_square = - 1 / upper_max_square
nu_lower_square = - 1 / lower_max_square

gamma_upper_square = nu_upper_square*upper_s_square
gamma_lower_square = nu_lower_square*lower_s_square

beta_upper_square = -nu_upper_square*lower_P_square
beta_lower_square = -nu_lower_square*upper_P_square

println("nu in [$(nu_lower_square), $(nu_upper_square)]")
println("gamma in [$(gamma_lower_square), $(gamma_upper_square)]")
println("beta in [$(beta_lower_square), $(beta_upper_square)]")

nu in [1.2700622364263507, 1.7445994602787034]
gamma in [2.1862551631415807, 3.1767687013855075]
beta in [0.07731208236743356, 0.13803229158418606]


### Triangular lattice

In [20]:
df_10_000_triangular = CSV.read("DataFrames/N10_000_q10_000_triangular.csv", DataFrame)
df_40_000_triangular = CSV.read("DataFrames/N40_000_q10_000_triangular.csv", DataFrame)
df_90_000_triangular = CSV.read("DataFrames/N90_000_q10_000_triangular.csv", DataFrame)
df_160_000_triangular = CSV.read("DataFrames/N160_000_q10_000_triangular.csv", DataFrame)
df_250_000_triangular = CSV.read("DataFrames/N250_000_q10_000_triangular.csv", DataFrame)
df_360_000_triangular = CSV.read("DataFrames/N360_000_q10_000_triangular.csv", DataFrame)
df_490_000_triangular = CSV.read("DataFrames/N490_000_q10_000_triangular.csv", DataFrame)
df_640_000_triangular = CSV.read("DataFrames/N640_000_q10_000_triangular.csv", DataFrame)
df_810_000_triangular = CSV.read("DataFrames/N810_000_q10_000_triangular.csv", DataFrame)
df_1_000_000_triangular = CSV.read("DataFrames/N1_000_000_q10_000_triangular.csv", DataFrame)

Row,q,P,chi,average_cluster_size
,Float64,Float64,Float64,Float64
1,0.0,6.00003e-12,0.0034641,1.00001
2,0.00010001,2.11476e-6,0.318672,1.0006
3,0.00020002,2.43699e-6,0.495977,1.0012
4,0.00030003,2.74803e-6,0.434105,1.0018
5,0.00040004,2.9602e-6,0.241209,1.0024
6,0.00050005,3.00372e-6,0.127404,1.00301
7,0.00060006,3.0298e-6,0.169831,1.00361
8,0.00070007,3.05388e-6,0.225621,1.00421
9,0.00080008,3.07687e-6,0.26625,1.00482


In [21]:
# Giant component, P_inf
println(2*sin(pi/18))
FSS_P_q_triangular = plot(df_10_000_triangular.q[2000:4000], df_10_000_triangular.P[2000:4000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=10_000", legend=:topleft)
plot!(FSS_P_q_triangular,df_40_000_triangular.q[2000:4000], df_40_000_triangular.P[2000:4000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=40_000", legend=:topleft)
plot!(FSS_P_q_triangular,df_90_000_triangular.q[2000:4000], df_90_000_triangular.P[2000:4000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=90_000", legend=:topleft)
plot!(FSS_P_q_triangular,df_160_000_triangular.q[2000:4000], df_160_000_triangular.P[2000:4000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=160_000", legend=:topleft)
plot!(FSS_P_q_triangular,df_250_000_triangular.q[2000:4000], df_250_000_triangular.P[2000:4000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=250_000", legend=:topleft)
plot!(FSS_P_q_triangular,df_360_000_triangular.q[2000:4000], df_360_000_triangular.P[2000:4000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=360_000", legend=:topleft)
plot!(FSS_P_q_triangular,df_490_000_triangular.q[2000:4000], df_490_000_triangular.P[2000:4000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=490_000", legend=:topleft)
plot!(FSS_P_q_triangular,df_640_000_triangular.q[2000:4000], df_640_000_triangular.P[2000:4000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=640_000", legend=:topleft)
plot!(FSS_P_q_triangular,df_810_000_triangular.q[2000:4000], df_810_000_triangular.P[2000:4000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=810_000", legend=:topleft)
plot!(FSS_P_q_triangular,df_1_000_000_triangular.q[2000:4000], df_1_000_000_triangular.P[2000:4000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=1_000_000", legend=:topleft)
savefig(FSS_P_q_triangular, "figures/FSS_P_q_triangular.svg")

0.34729635533386066


"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_P_q_triangular.svg"

In [22]:
ksi_triangular = round.(Int, sqrt.([10_000, 40_000, 90_000, 160_000, 250_000, 360_000, 490_000, 640_000, 810_000, 1_000_000]))
df_P_triangular = DataFrame(q = df_10_000_triangular.q)  # Initialize with q values (common for all N)
df_P_triangular[!, "P_ksi_$(ksi_triangular[1])"] = df_10_000_triangular.P
df_P_triangular[!, "P_ksi_$(ksi_triangular[2])"] = df_40_000_triangular.P
df_P_triangular[!, "P_ksi_$(ksi_triangular[3])"] = df_90_000_triangular.P
df_P_triangular[!, "P_ksi_$(ksi_triangular[4])"] = df_160_000_triangular.P
df_P_triangular[!, "P_ksi_$(ksi_triangular[5])"] = df_250_000_triangular.P
df_P_triangular[!, "P_ksi_$(ksi_triangular[6])"] = df_360_000_triangular.P
df_P_triangular[!, "P_ksi_$(ksi_triangular[7])"] = df_490_000_triangular.P
df_P_triangular[!, "P_ksi_$(ksi_triangular[8])"] = df_640_000_triangular.P
df_P_triangular[!, "P_ksi_$(ksi_triangular[9])"] = df_810_000_triangular.P
df_P_triangular[!, "P_ksi_$(ksi_triangular[10])"] = df_1_000_000_triangular.P
# display(df_P_triangular)
# df_P_triangular = df_P_triangular[2:end-1, :]  # Remove NaN in the first and last row due to NaN

10000-element Vector{Float64}:
 6.00002700005401e-12
 2.1147566121600132e-6
 2.436986832898198e-6
 2.748026485849321e-6
 2.960197805369211e-6
 3.00371866440192e-6
 3.0298027022466327e-6
 3.053879991741737e-6
 3.0768700089487046e-6
 3.1045971865694387e-6
 ⋮
 1.0000059989626744
 1.0000059968886639
 1.0000059974057525
 1.0000059970572075
 1.0000059982310368
 1.0000059997829867
 1.0000059972867104
 1.0000059997427893
 1.0000059989373953

In [23]:
r2_arr_P_triangular, model_arr_P_triangular, slope_arr_P_triangular = line_fitter(df_P_triangular, ksi_triangular)

([0.7460254714487825, 0.9999019467095361, 0.9993432119650721, 0.9990087454966049, 0.9990471227376164, 0.9994729413085717, 0.99966194021955, 0.9997303616185909, 0.9996996588327484, 0.9995681684238863  …  0.7459759963590227, 0.7460675352045405, 0.7461447248183823, 0.746086358553014, 0.7460679481054382, 0.7460393878079838, 0.7459643282438531, 0.7461244368889863, 0.7460273753350619, 0.7460379714767421], Any[StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

log_param ~ 1 + log_ksi

Coefficients:
─────────────────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error             t  Pr(>|t|)     Lower 95%     Upper 95%
─────────────────────────────────────────────────────────────────────────────────────────
(Intercept)  -25.8393      2.30072e-6  -11230929.40    <1e-53  -25.8393      -25.8393
log_k

In [24]:
FSS_P_R2_triangular = plot(df_P_triangular.q, r2_arr_P_triangular, xlabel="q", xticks=0:0.1:1, ylabel=L"$R^2$-coefficient", color=:green, legend=false)
savefig(FSS_P_R2_triangular, "figures/FFS_P_R2_triangular.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FFS_P_R2_triangular.svg"

In [25]:
i_start = 3400
i_end = 3800
best_index_P_triangular = i_start+argmax(r2_arr_P_triangular[i_start:i_end]) - 1 
println(best_index_P_triangular)
plot(df_P_triangular.q[i_start:i_end], r2_arr_P_triangular[i_start:i_end], xlabel="q", xticks=0.3:0.005:0.4, yticks=0:0.1:1, ylabel=L"$R^2$-coefficient", color=:green, label=L"Triangular: $R^2$")
vline!([df_P_triangular.q[best_index_P_triangular]], color=:red, linestyle=:dash, label=L"$p_c$")
savefig("figures/FFS_P_R2_triangular_zoomed.svg")

3462


"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FFS_P_R2_triangular_zoomed.svg"

In [26]:
p_c_triangular = df_P_triangular.q[best_index_P_triangular]
println("p_c: ", p_c_triangular)

p_c: 0.3461346134613461


In [27]:
display(model_arr_P_triangular[best_index_P_triangular])

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

log_param ~ 1 + log_ksi

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)   0.374526   0.0745899    5.02    0.0010   0.202521   0.546531
log_ksi      -0.196836   0.0121185  -16.24    <1e-06  -0.224781  -0.16889
──────────────────────────────────────────────────────────────────────────

In [28]:
# Average cluster size, <s>
FSS_s_triangular = plot(df_10_000_triangular.q[3000:3800], df_10_000_triangular.average_cluster_size[3000:3800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=10_000", legend=:topright)
plot!(FSS_s_triangular,df_40_000_triangular.q[3000:3800], df_40_000_triangular.average_cluster_size[3000:3800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=40_000")
plot!(FSS_s_triangular,df_90_000_triangular.q[3000:3800], df_90_000_triangular.average_cluster_size[3000:3800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=90_000")
plot!(FSS_s_triangular,df_160_000_triangular.q[3000:3800], df_160_000_triangular.average_cluster_size[3000:3800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=160_000")
plot!(FSS_s_triangular,df_250_000_triangular.q[3000:3800], df_250_000_triangular.average_cluster_size[3000:3800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=250_000")
plot!(FSS_s_triangular,df_360_000_triangular.q[3000:3800], df_360_000_triangular.average_cluster_size[3000:3800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=360_000")
plot!(FSS_s_triangular,df_490_000_triangular.q[3000:3800], df_490_000_triangular.average_cluster_size[3000:3800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=490_000")
plot!(FSS_s_triangular,df_640_000_triangular.q[3000:3800], df_640_000_triangular.average_cluster_size[3000:3800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=640_000")
plot!(FSS_s_triangular,df_810_000_triangular.q[3000:3800], df_810_000_triangular.average_cluster_size[3000:3800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=810_000")
plot!(FSS_s_triangular,df_1_000_000_triangular.q[3000:3800], df_1_000_000_triangular.average_cluster_size[3000:3800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=1_000_000")
savefig(FSS_s_triangular, "figures/FSS_s_triangular.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_s_triangular.svg"

In [35]:
df_s_triangular = DataFrame(q = df_10_000_triangular.q)  # Initialize with q values (common for all N)
df_s_triangular[!, "s_ksi_$(ksi_triangular[1])"] = df_10_000_triangular.average_cluster_size
df_s_triangular[!, "s_ksi_$(ksi_triangular[2])"] = df_40_000_triangular.average_cluster_size
df_s_triangular[!, "s_ksi_$(ksi_triangular[3])"] = df_90_000_triangular.average_cluster_size
df_s_triangular[!, "s_ksi_$(ksi_triangular[4])"] = df_160_000_triangular.average_cluster_size
df_s_triangular[!, "s_ksi_$(ksi_triangular[5])"] = df_250_000_triangular.average_cluster_size
df_s_triangular[!, "s_ksi_$(ksi_triangular[6])"] = df_360_000_triangular.average_cluster_size
df_s_triangular[!, "s_ksi_$(ksi_triangular[7])"] = df_490_000_triangular.average_cluster_size
df_s_triangular[!, "s_ksi_$(ksi_triangular[8])"] = df_640_000_triangular.average_cluster_size
df_s_triangular[!, "s_ksi_$(ksi_triangular[9])"] = df_810_000_triangular.average_cluster_size
df_s_triangular[!, "s_ksi_$(ksi_triangular[10])"] = df_1_000_000_triangular.average_cluster_size

# df_s_triangular = df_s_triangular[2:end-1, :]  # Remove NaN in the first row

max_vec = zeros(length(ksi_triangular))
for i in 1:length(ksi_triangular)
    max_vec[i] = maximum(df_s_triangular[!, "s_ksi_$(ksi[i])"])
end
println(max_vec)
df_sm_triangular = DataFrame(ksi=ksi_triangular)  # Initialize with q values (common for all N)
df_sm_triangular[!, "max_s"] = max_vec

display(df_sm_triangular)

Row,ksi,max_s
,Int64,Float64
1,100,302.113
2,200,1041.79
3,300,2272.75
4,400,3421.93
5,500,5577.58
6,600,6668.53
7,700,9695.59
8,800,11867.5
9,900,13868.2


[302.1131229509579, 1041.7914603275783, 2272.75163757948, 3421.9318032661417, 5577.575710426174, 6668.526215506095, 9695.59489912522, 11867.523720728574, 13868.170360653847, 16039.143425476692]


In [36]:
model_sm_triangular = lm(@formula(log(max_s) ~ log(ksi) ), df_sm_triangular)
println(r2(model_sm_triangular))
display(model_sm_triangular)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

:(log(max_s)) ~ 1 + :(log(ksi))

Coefficients:
─────────────────────────────────────────────────────────────────────────
                Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────
(Intercept)  -2.2345    0.156002   -14.32    <1e-06   -2.59424   -1.87476
log(ksi)      1.73464   0.0253454   68.44    <1e-11    1.6762     1.79309
─────────────────────────────────────────────────────────────────────────

0.9982949869455814


In [37]:
FSS_sm_triangular = scatter(log.(df_sm_triangular.ksi), log.(df_sm_triangular.max_s), xlabel=L"log(\xi)", ylabel=L"log(max(\langle s \rangle))", color=:green, label=L"Triangular: $max(\langle s \rangle)$")
plot!(FSS_sm_triangular, log.(df_sm_triangular.ksi), predict(model_sm_triangular), color=:red, label="Best fit")
annotate!(FSS_sm_triangular, 4.9, 8.95, text("R^2: $(round(r2(model_sm_triangular),digits=4))",8))
savefig(FSS_sm_triangular, "figures/FSS_sm_triangular.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_sm_triangular.svg"

In [38]:
# i_start = 3000
# i_end = 4000
# best_index_s_triangular = i_start+argmax(r2_arr_s[i_start:i_end])
# println(best_index_s_triangular)
# plot(df_s_triangular.q[i_start:i_end], r2_arr_s_triangular[i_start:i_end], xlabel="q", xticks=0.3:0.01:0.4, ylabel=L"$R^2$-coefficient", color=:green, label=L"$R^2$", legend=:topright)
# vline!([df_s_triangular.q[best_index_s_triangular]], color=:red, linestyle=:dash, label=L"$p_c$")

In [39]:
# display(model_arr_s_triangular[best_index_s_triangular])

In [40]:
# Susceptibility, chi

FSS_chi_triangular = plot(df_10_000_triangular.q[3000:4000], df_10_000_triangular.chi[3000:4000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=10_000", legend=:topright)
plot!(FSS_chi_triangular, df_40_000_triangular.q[3000:4000], df_40_000_triangular.chi[3000:4000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=40_000")
plot!(FSS_chi_triangular,df_90_000_triangular.q[3000:4000], df_90_000_triangular.chi[3000:4000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=90_000")
plot!(FSS_chi_triangular,df_160_000_triangular.q[3000:4000], df_160_000_triangular.chi[3000:4000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=160_000")
plot!(FSS_chi_triangular,df_250_000_triangular.q[3000:4000], df_250_000_triangular.chi[3000:4000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=250_000")
plot!(FSS_chi_triangular,df_360_000_triangular.q[3000:4000], df_360_000_triangular.chi[3000:4000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=360_000")
plot!(FSS_chi_triangular,df_490_000_triangular.q[3000:4000], df_490_000_triangular.chi[3000:4000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=490_000")
plot!(FSS_chi_triangular,df_640_000_triangular.q[3000:4000], df_640_000_triangular.chi[3000:4000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=640_000")
plot!(FSS_chi_triangular,df_810_000_triangular.q[3000:4000], df_810_000_triangular.chi[3000:4000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=810_000")
plot!(FSS_chi_triangular,df_1_000_000_triangular.q[3000:4000], df_1_000_000_triangular.chi[3000:4000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=1_000_000")
savefig(FSS_chi_triangular, "figures/FSS_chi_triangular.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_chi_triangular.svg"

In [41]:
q_max = zeros(length(ksi_triangular))
for i in 1:length(ksi_triangular)
    q_max[i] = df_s_triangular.q[argmax(df_s_triangular[!, "s_ksi_$(ksi[i])"])]
end
println(q_max)


df_max_triangular = DataFrame(ksi=ksi_triangular)  # Initialize with q values (common for all N)
df_max_triangular[!, "q_max"] = abs.(q_max.- p_c_triangular)
display(df_max_triangular)
# r2_arr_max_triangular, model_arr_max_triangular, slope_arr_max_triangular = line_fitter(df_max_triangular, ksi)


Row,ksi,q_max
,Int64,Float64
1,100,0.0133013
2,200,0.00660066
3,300,0.00370037
4,400,0.00190019
5,500,0.00180018
6,600,0.00130013
7,700,0.0010001
8,800,0.00120012
9,900,0.00210021


[0.33283328332833284, 0.33953395339533954, 0.3424342434243424, 0.34423442344234423, 0.34433443344334436, 0.34483448344834483, 0.34513451345134516, 0.34493449344934496, 0.344034403440344, 0.34543454345434543]


In [42]:
model_max_triangular = lm(@formula(log(q_max) ~ log(ksi) ), df_max_triangular)
println(r2(model_max_triangular))
display(model_max_triangular)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

:(log(q_max)) ~ 1 + :(log(ksi))

Coefficients:
─────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────
(Intercept)   0.966983    0.915178   1.06    0.3216   -1.14342   3.07739
log(ksi)     -1.15939     0.148688  -7.80    <1e-04   -1.50226  -0.816512
─────────────────────────────────────────────────────────────────────────

0.8837212197659398


In [50]:
FSS_max_triangular = scatter(log.(ksi_triangular), log.(df_max_triangular.q_max), xlabel=L"log(\xi)", ylabel=L"\left|q_{max}-p_c\right|",color=:green, label="Triangular: q_max",legend=:topright)
plot!(FSS_max_triangular, log.(ksi_triangular), predict(model_max_triangular), color=:red, label="Best fit")
annotate!(FSS_max_triangular, 6.55, -4.73, text("R^2: $(round(r2(model_max_triangular),digits=4))",8))
savefig(FSS_max_triangular, "figures/FSS_max_triangular.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_max_triangular.svg"

In [51]:
# Upper and lower bounds
# Upper and lower bounds

lower_P_triangular = coeftable(model_arr_P_triangular[best_index_P_triangular]).cols[5][2]
upper_P_triangular = coeftable(model_arr_P_triangular[best_index_P_triangular]).cols[6][2]

lower_s_triangular = coeftable(model_sm_triangular).cols[5][2]
upper_s_triangular = coeftable(model_sm_triangular).cols[6][2]

lower_max_triangular = coeftable(model_max_triangular).cols[5][2]
upper_max_triangular = coeftable(model_max_triangular).cols[6][2]

println(lower_P_triangular, " ", upper_P_triangular, " ", lower_s_triangular, " ", upper_s_triangular, " ", lower_max_triangular, " ", upper_max_triangular)

-0.2247811460731491 -0.1688903188588484 1.6761953080308374 1.7930885302484738 -1.5022627097425922 -0.8165120371665132


In [52]:
nu_triangular = - 1 / coef(model_max_triangular)[2]
gamma_triangular = nu_triangular*coef(model_sm_triangular)[2]
beta_triangular = -nu_triangular*slope_arr_P_triangular[best_index_P_triangular]
println("Numerical: nu = $(nu_triangular), gamma = $(gamma_triangular), beta = $(beta_triangular)")
println("Analytical: nu = $(4/3), gamma = $(43/18), beta = $(5/36)")

Numerical: nu = 0.8625244874112814, gamma = 1.4961711321480529, beta = 0.1697756392494597
Analytical: nu = 1.3333333333333333, gamma = 2.388888888888889, beta = 0.1388888888888889


In [49]:
nu_upper_triangular = - 1 / upper_max_triangular
nu_lower_triangular = - 1 / lower_max_triangular

gamma_upper_triangular = nu_upper_triangular*upper_s_triangular
gamma_lower_triangular = nu_lower_triangular*lower_s_triangular

beta_upper_triangular = -nu_upper_triangular*lower_P_triangular
beta_lower_triangular = -nu_lower_triangular*upper_P_triangular

println("nu in [$(nu_lower_triangular), $(nu_upper_triangular)]")
println("gamma in [$(gamma_lower_triangular), $(gamma_upper_triangular)]")
println("beta in [$(beta_lower_triangular), $(beta_upper_triangular)]")

nu in [0.6656625326014693, 1.2247216874722786]
gamma in [1.1157804138785072, 2.1960344105530987]
beta in [0.11242395738345072, 0.27529434453065993]


### Honeycomb

In [53]:
df_10_000_honeycomb = CSV.read("DataFrames/N10_000_q10_000_honeycomb.csv", DataFrame)
df_40_000_honeycomb = CSV.read("DataFrames/N40_000_q10_000_honeycomb.csv", DataFrame)
df_90_000_honeycomb = CSV.read("DataFrames/N90_000_q10_000_honeycomb.csv", DataFrame)
df_160_000_honeycomb = CSV.read("DataFrames/N160_000_q10_000_honeycomb.csv", DataFrame)
df_250_000_honeycomb = CSV.read("DataFrames/N250_000_q10_000_honeycomb.csv", DataFrame)
df_360_000_honeycomb = CSV.read("DataFrames/N360_000_q10_000_honeycomb.csv", DataFrame)
# df_490_000_honeycomb = CSV.read("DataFrames/N490_000_q10_000_honeycomb.csv", DataFrame)
df_640_000_honeycomb = CSV.read("DataFrames/N640_000_q10_000_honeycomb.csv", DataFrame)
df_810_000_honeycomb = CSV.read("DataFrames/N810_000_q10_000_honeycomb.csv", DataFrame)
df_1_000_000_honeycomb = CSV.read("DataFrames/N1_000_000_q10_000_honeycomb.csv", DataFrame)

Row,q,P,chi,average_cluster_size
,Float64,Float64,Float64,Float64
1,0.0,3.00001e-12,0.00244949,1.0
2,0.00010001,2.03075e-6,0.172574,1.0003
3,0.00020002,2.14887e-6,0.355934,1.0006
4,0.00030003,2.25426e-6,0.435425,1.0009
5,0.00040004,2.43486e-6,0.495721,1.0012
6,0.00050005,2.58883e-6,0.492028,1.0015
7,0.00060006,2.71965e-6,0.467258,1.0018
8,0.00070007,2.82172e-6,0.408034,1.0021
9,0.00080008,2.87637e-6,0.3823,1.0024


In [54]:
println(1-2*sin(pi/18))
FSS_P_q_honeycomb = plot(df_10_000_honeycomb.q[5000:7000], df_10_000_honeycomb.P[5000:7000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=10_000", legend=:topleft)
plot!(FSS_P_q_honeycomb, df_40_000_honeycomb.q[5000:7000], df_40_000_honeycomb.P[5000:7000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=40_000", legend=:topleft)
plot!(FSS_P_q_honeycomb,df_90_000_honeycomb.q[5000:7000], df_90_000_honeycomb.P[5000:7000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=90_000", legend=:topleft)
plot!(FSS_P_q_honeycomb,df_160_000_honeycomb.q[5000:7000], df_160_000_honeycomb.P[5000:7000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=160_000", legend=:topleft)
plot!(FSS_P_q_honeycomb,df_250_000_honeycomb.q[5000:7000], df_250_000_honeycomb.P[5000:7000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=250_000", legend=:topleft)
plot!(FSS_P_q_honeycomb,df_360_000_honeycomb.q[5000:7000], df_360_000_honeycomb.P[5000:7000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=360_000", legend=:topleft)
# plot!(df_490_000_honeycomb.q[5000:7000], df_490_000_honeycomb.P[5000:7000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=490_000", legend=:topleft)
plot!(FSS_P_q_honeycomb,df_640_000_honeycomb.q[5000:7000], df_640_000_honeycomb.P[5000:7000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=640_000", legend=:topleft)
plot!(FSS_P_q_honeycomb,df_810_000_honeycomb.q[5000:7000], df_810_000_honeycomb.P[5000:7000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=810_000", legend=:topleft)
plot!(FSS_P_q_honeycomb,df_1_000_000_honeycomb.q[5000:7000], df_1_000_000_honeycomb.P[5000:7000], xlabel="q", xticks=(0:0.02:1.0), ylabel=L"\langle P_\infty\rangle", label="N=1_000_000", legend=:topleft)
# vline!(FSS_P_q_honeycomb,[1-2*sin(pi/18)], color=:red, linestyle=:dash, label=L"$p_c$")
savefig(FSS_P_q_honeycomb, "figures/FSS_P_q_honeycomb.svg")

0.6527036446661394


"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_P_q_honeycomb.svg"

In [56]:
ksi_honeycomb = round.(Int, sqrt.([10_000, 40_000, 90_000, 160_000, 250_000, 360_000, 640_000, 810_000, 1_000_000]))
df_P_honeycomb = DataFrame(q = df_10_000_honeycomb.q)  # Initialize with q values (common for all N)
df_P_honeycomb[!, "P_ksi_$(ksi_honeycomb[1])"] = df_10_000_honeycomb.P
df_P_honeycomb[!, "P_ksi_$(ksi_honeycomb[2])"] = df_40_000_honeycomb.P
df_P_honeycomb[!, "P_ksi_$(ksi_honeycomb[3])"] = df_90_000_honeycomb.P
df_P_honeycomb[!, "P_ksi_$(ksi_honeycomb[4])"] = df_160_000_honeycomb.P
df_P_honeycomb[!, "P_ksi_$(ksi_honeycomb[5])"] = df_250_000_honeycomb.P
df_P_honeycomb[!, "P_ksi_$(ksi_honeycomb[6])"] = df_360_000_honeycomb.P
# df_P_honeycomb[!, "P_ksi_$(ksi_honeycomb[7])"] = df_490_000_honeycomb.P
df_P_honeycomb[!, "P_ksi_$(ksi_honeycomb[7])"] = df_640_000_honeycomb.P
df_P_honeycomb[!, "P_ksi_$(ksi_honeycomb[8])"] = df_810_000_honeycomb.P
df_P_honeycomb[!, "P_ksi_$(ksi_honeycomb[9])"] = df_1_000_000_honeycomb.P
# display(df_P_honeycomb)
# df_P_honeycomb = df_P_honeycomb[2:end-1, :]  # Remove NaN in the first and last row

10000-element Vector{Float64}:
 3.000006750003375e-12
 2.0307450059200993e-6
 2.1488691666980746e-6
 2.254262071261972e-6
 2.4348605583139113e-6
 2.588826967570546e-6
 2.719647010677471e-6
 2.821716503425301e-6
 2.876369811042086e-6
 2.918501498375274e-6
 ⋮
 1.0000029996271655
 1.0000029985746548
 1.0000029989941694
 1.0000029986507322
 1.0000029992555866
 1.0000030000600217
 1.0000029987739931
 1.0000030000005138
 1.0000029995967568

In [57]:
r2_arr_P_honeycomb, model_arr_P_honeycomb, slope_arr_P_honeycomb = line_fitter(df_P_honeycomb, ksi_honeycomb)

([0.7485653147668857, 0.9984507863931583, 0.9999028127134546, 0.9998199276536959, 0.9995470188991626, 0.9992429154565834, 0.9990936688785709, 0.9989894096322807, 0.9990002486866856, 0.9991690824129  …  0.74857134607794, 0.7486087370993857, 0.7486914082998981, 0.7486173656363304, 0.7486414057582895, 0.7486025939075298, 0.7485271812315513, 0.7486725974670028, 0.7485899450642847, 0.7486033315986129], Any[StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

log_param ~ 1 + log_ksi

Coefficients:
─────────────────────────────────────────────────────────────────────────────────────────
                    Coef.  Std. Error             t  Pr(>|t|)     Lower 95%     Upper 95%
─────────────────────────────────────────────────────────────────────────────────────────
(Intercept)  -26.5324      1.23466e-6  -21489671.31    <1e-48  -26.5324      -26.5324
log_ksi

In [58]:
FSS_P_R2_honeycomb = plot(df_P_honeycomb.q, r2_arr_P_honeycomb, xlabel="q", xticks=0:0.1:1, ylabel=L"$R^2$-coefficient", color=:green, legend=false)
savefig(FSS_P_R2_honeycomb, "figures/FFS_P_R2_honeycomb.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FFS_P_R2_honeycomb.svg"

In [59]:
i_start = 6440
i_end = 7000
best_index_P_honeycomb = i_start+argmax(r2_arr_P_honeycomb[i_start:i_end]) - 1
println(best_index_P_honeycomb)
FSS_P_R2_honeycomb_zoomed = plot(df_P_honeycomb.q[i_start:i_end], r2_arr_P_honeycomb[i_start:i_end], xlabel="q", xticks=0:0.005:1, ylabel=L"$R^2$-coefficient", label=L"Honeycomb: $R^2$", color=:green)
vline!(FSS_P_R2_honeycomb_zoomed, [df_P_honeycomb.q[best_index_P_honeycomb]], color=:red, linestyle=:dash, label=L"$p_c$")
savefig(FSS_P_R2_honeycomb_zoomed, "figures/FFS_P_R2_honeycomb_zoomed.svg")

6519


"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FFS_P_R2_honeycomb_zoomed.svg"

In [60]:
p_c_honeycomb = df_P_honeycomb.q[best_index_P_honeycomb]
println("p_c: ", p_c_honeycomb)

p_c: 0.6518651865186519


In [61]:
display(model_arr_P_honeycomb[best_index_P_honeycomb])

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

log_param ~ 1 + log_ksi

Coefficients:
────────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error       t  Pr(>|t|)    Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────────
(Intercept)   0.114872  0.0495562     2.32    0.0536  -0.00230986   0.232054
log_ksi      -0.136222  0.00811142  -16.79    <1e-06  -0.155403    -0.117042
────────────────────────────────────────────────────────────────────────────

In [62]:
# FSS - s

FSS_s_honeycomb = plot(df_10_000_honeycomb.q[6000:6800], df_10_000_honeycomb.average_cluster_size[6000:6800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=10_000", legend=:topright)
plot!(FSS_s_honeycomb, df_40_000_honeycomb.q[6000:6800], df_40_000_honeycomb.average_cluster_size[6000:6800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=40_000")
plot!(FSS_s_honeycomb,df_90_000_honeycomb.q[6000:6800], df_90_000_honeycomb.average_cluster_size[6000:6800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=90_000")
plot!(FSS_s_honeycomb,df_160_000_honeycomb.q[6000:6800], df_160_000_honeycomb.average_cluster_size[6000:6800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=160_000")
plot!(FSS_s_honeycomb,df_250_000_honeycomb.q[6000:6800], df_250_000_honeycomb.average_cluster_size[6000:6800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=250_000")
plot!(FSS_s_honeycomb,df_360_000_honeycomb.q[6000:6800], df_360_000_honeycomb.average_cluster_size[6000:6800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=360_000")
# plot!(df_490_000_honeycomb.q[6000:6800], df_490_000_honeycomb.average_cluster_size[6000:6800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=490_000")
plot!(FSS_s_honeycomb,df_640_000_honeycomb.q[6000:6800], df_640_000_honeycomb.average_cluster_size[6000:6800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=640_000")
plot!(FSS_s_honeycomb,df_810_000_honeycomb.q[6000:6800], df_810_000_honeycomb.average_cluster_size[6000:6800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=810_000")
plot!(FSS_s_honeycomb,df_1_000_000_honeycomb.q[6000:6800], df_1_000_000_honeycomb.average_cluster_size[6000:6800], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\langle s\rangle", label="N=1_000_000")
savefig(FSS_s_honeycomb, "figures/FSS_s_honeycomb.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_s_honeycomb.svg"

In [63]:
df_s_honeycomb = DataFrame(q = df_10_000_honeycomb.q)  # Initialize with q values (common for all N)
df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[1])"] = df_10_000_honeycomb.average_cluster_size
df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[2])"] = df_40_000_honeycomb.average_cluster_size
df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[3])"] = df_90_000_honeycomb.average_cluster_size
df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[4])"] = df_160_000_honeycomb.average_cluster_size
df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[5])"] = df_250_000_honeycomb.average_cluster_size
df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[6])"] = df_360_000_honeycomb.average_cluster_size
# df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[7])"] = df_490_000_honeycomb.average_cluster_size
df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[7])"] = df_640_000_honeycomb.average_cluster_size
df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[8])"] = df_810_000_honeycomb.average_cluster_size
df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[9])"] = df_1_000_000_honeycomb.average_cluster_size

# df_s_honeycomb = df_s_honeycomb[2:end-1, :]  # Remove NaN in the first row

max_vec = zeros(length(ksi_honeycomb))
for i in 1:length(ksi_honeycomb)
    max_vec[i] = maximum(df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[i])"])
end
println(max_vec)
df_sm_honeycomb = DataFrame(ksi=ksi_honeycomb)  # Initialize with q values (common for all N)
df_sm_honeycomb[!, "max_s"] = max_vec

display(df_sm_honeycomb)

Row,ksi,max_s
,Int64,Float64
1,100,379.726
2,200,1286.85
3,300,2664.06
4,400,4099.33
5,500,5758.31
6,600,8767.9
7,800,14503.0
8,900,16951.0
9,1000,20981.0


[379.7264140806954, 1286.853300175229, 2664.0640287917176, 4099.334860078835, 5758.306710122759, 8767.901771262437, 14503.034783519157, 16950.953773392386, 20980.970405609954]


In [64]:
model_sm_honeycomb = lm(@formula(log(max_s) ~ log(ksi) ), df_sm_honeycomb)
println(r2(model_sm_honeycomb))
display(model_sm_honeycomb)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

:(log(max_s)) ~ 1 + :(log(ksi))

Coefficients:
─────────────────────────────────────────────────────────────────────────
                Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────
(Intercept)  -2.04464   0.111021   -18.42    <1e-06   -2.30716   -1.78211
log(ksi)      1.73457   0.0181721   95.45    <1e-11    1.6916     1.77754
─────────────────────────────────────────────────────────────────────────

0.9992322955623639


In [65]:
println("R^2: ", r2(model_sm_honeycomb))
FFS_sm_honeycomb = scatter(log.(df_sm_honeycomb.ksi), log.(df_sm_honeycomb.max_s), xlabel=L"log(\xi)", ylabel=L"log(max(\langle s \rangle))", color=:green, label=L"Honeycomb: $max(\langle s \rangle)$")
plot!(FFS_sm_honeycomb, log.(df_sm_honeycomb.ksi), predict(model_sm_honeycomb), color=:red, label="Best fit")
annotate!(FFS_sm_honeycomb, 4.85, 9.13, text("R^2: $(round(r2(model_sm_honeycomb),digits=4))",8))
savefig(FFS_sm_honeycomb, "figures/FFS_sm_honeycomb.svg")

R^2: 0.9992322955623639


"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FFS_sm_honeycomb.svg"

In [68]:
# FSS - chi

FSS_chi_honeycomb = plot(df_10_000_honeycomb.q[6000:7000], df_10_000_honeycomb.chi[6000:7000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=10_000", legend=:topright)
plot!(FSS_chi_honeycomb, df_40_000_honeycomb.q[6000:7000], df_40_000_honeycomb.chi[6000:7000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=40_000")
plot!(FSS_chi_honeycomb,df_90_000_honeycomb.q[6000:7000], df_90_000_honeycomb.chi[6000:7000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=90_000")
plot!(FSS_chi_honeycomb,df_160_000_honeycomb.q[6000:7000], df_160_000_honeycomb.chi[6000:7000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=160_000")
plot!(FSS_chi_honeycomb,df_250_000_honeycomb.q[6000:7000], df_250_000_honeycomb.chi[6000:7000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=250_000")
plot!(FSS_chi_honeycomb,df_360_000_honeycomb.q[6000:7000], df_360_000_honeycomb.chi[6000:7000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=360_000")
# plot!(df_490_000_honeycomb.q[6000:7000], df_490_000_honeycomb.chi[6000:7000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=490_000")
plot!(FSS_chi_honeycomb,df_640_000_honeycomb.q[6000:7000], df_640_000_honeycomb.chi[6000:7000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=640_000")
plot!(FSS_chi_honeycomb,df_810_000_honeycomb.q[6000:7000], df_810_000_honeycomb.chi[6000:7000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=810_000")
plot!(FSS_chi_honeycomb,df_1_000_000_honeycomb.q[6000:7000], df_1_000_000_honeycomb.chi[6000:7000], xlabel="q", xticks=(0:0.01:1.0), ylabel=L"\chi", label="N=1_000_000")
savefig(FSS_chi_honeycomb, "figures/FSS_chi_honeycomb.svg")

"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_chi_honeycomb.svg"

In [69]:
q_max = zeros(length(ksi_honeycomb))
for i in 1:length(ksi_honeycomb)
    q_max[i] = df_s_honeycomb.q[argmax(df_s_honeycomb[!, "s_ksi_$(ksi_honeycomb[i])"])]
end
println(q_max)


df_max_honeycomb = DataFrame(ksi=ksi_honeycomb)  # Initialize with q values (common for all N)
df_max_honeycomb[!, "q_max"] = abs.(q_max.- p_c_honeycomb)
display(df_max_honeycomb)
# r2_arr_max_honeycomb, model_arr_max_honeycomb, slope_arr_max_honeycomb = line_fitter(df_max_honeycomb, ksi)



Row,ksi,q_max
,Int64,Float64
1,100,0.0141014
2,200,0.00610061
3,300,0.00530053
4,400,0.0040004
5,500,0.00420042
6,600,0.0030003
7,800,0.00280028
8,900,0.00220022
9,1000,0.00230023


[0.6377637763776378, 0.6457645764576457, 0.6465646564656465, 0.6478647864786479, 0.6476647664766476, 0.6488648864886488, 0.6490649064906491, 0.6496649664966496, 0.6495649564956496]


In [70]:
model_max_honeycomb = lm(@formula(log(q_max) ~ log(ksi) ), df_max_honeycomb)
println(r2(model_max_honeycomb))
display(model_max_honeycomb)


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

:(log(q_max)) ~ 1 + :(log(ksi))

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error       t  Pr(>|t|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  -0.930657   0.331101    -2.81    0.0261  -1.71359   -0.147727
log(ksi)     -0.752675   0.0541951  -13.89    <1e-05  -0.880826  -0.624524
──────────────────────────────────────────────────────────────────────────

0.9649795196129254


In [71]:
println("R^2: ", r2(model_max_honeycomb))
FSS_max_honeycomb = scatter(log.(ksi_honeycomb), log.(df_max_honeycomb.q_max), xlabel=L"log(\xi)", ylabel=L"log(\left|q_{max}-p_c\right|)",color=:green, label="Honeycomb: q_max",legend=:topright)
plot!(FSS_max_honeycomb, log.(ksi_honeycomb), predict(model_max_honeycomb), color=:red, label="Best fit")
annotate!(FSS_max_honeycomb, 6.4, -4.55, text("R^2: $(round(r2(model_max_honeycomb),digits=4))",8))
savefig(FSS_max_honeycomb, "figures/FSS_max_honeycomb.svg")

R^2: 0.9649795196129254


"c:\\Users\\jonas\\Documents\\Codespace\\TFY4235\\Numerical_Physics\\Project_Percolation\\figures\\FSS_max_honeycomb.svg"

In [72]:
# Upper and lower bounds
# Upper and lower bounds

lower_P_honeycomb = coeftable(model_arr_P_honeycomb[best_index_P_honeycomb]).cols[5][2]
upper_P_honeycomb = coeftable(model_arr_P_honeycomb[best_index_P_honeycomb]).cols[6][2]

lower_s_honeycomb = coeftable(model_sm_honeycomb).cols[5][2]
upper_s_honeycomb = coeftable(model_sm_honeycomb).cols[6][2]

lower_max_honeycomb = coeftable(model_max_honeycomb).cols[5][2]
upper_max_honeycomb = coeftable(model_max_honeycomb).cols[6][2]

println(lower_P_honeycomb, " ", upper_P_honeycomb, " ", lower_s_honeycomb, " ", upper_s_honeycomb, " ", lower_max_honeycomb, " ", upper_max_honeycomb)

-0.1554029583402364 -0.11704202303488329 1.69159774318693 1.7775383165996623 -0.8808259550586748 -0.6245236591650145


In [75]:
nu_honeycomb = - 1 / coef(model_max_honeycomb)[2]
gamma_honeycomb = nu_honeycomb*coef(model_sm_honeycomb)[2]
beta_honeycomb = -nu_honeycomb*slope_arr_P_honeycomb[best_index_P_honeycomb]
println("Numerical: nu = $(nu_honeycomb), gamma = $(gamma_honeycomb), beta = $(beta_honeycomb)")
println("Analytical: nu = $(4/3), gamma = $(43/18), beta = $(5/36)")

Numerical: nu = 1.3285950194575913, gamma = 2.3045384454265996, beta = 0.1809845226656001
Analytical: nu = 1.3333333333333333, gamma = 2.388888888888889, beta = 0.1388888888888889


In [76]:
nu_upper_honeycomb = - 1 / upper_max_honeycomb
nu_lower_honeycomb = - 1 / lower_max_honeycomb

gamma_upper_honeycomb = nu_upper_honeycomb*upper_s_honeycomb
gamma_lower_honeycomb = nu_lower_honeycomb*lower_s_honeycomb

beta_upper_honeycomb = -nu_upper_honeycomb*lower_P_honeycomb
beta_lower_honeycomb = -nu_lower_honeycomb*upper_P_honeycomb

println("nu in [$(nu_lower_honeycomb), $(nu_upper_honeycomb)]")
println("gamma in [$(gamma_lower_honeycomb), $(gamma_upper_honeycomb)]")
println("beta in [$(beta_lower_honeycomb), $(beta_upper_honeycomb)]")

nu in [1.135298062298115, 1.6012203626312504]
gamma in [1.920467640027986, 2.8462305478966536]
beta in [0.13287758195895433, 0.24883438130752245]


In [77]:
println(nu_honeycomb - nu_upper_honeycomb, " ", nu_honeycomb - nu_lower_honeycomb)

-0.27262534317365916 0.19329695715947626


In [78]:
println((nu_upper_triangular - nu_lower_triangular)/2)
println(nu_lower_triangular + (nu_upper_triangular - nu_lower_triangular)/2)
# println(5/36)

0.27952957743540463
0.945192110036874
